# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df=pd.read_csv("../output_data/cities.csv")
weather_df

,Unnamed: 0,City,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Vaini,75,88,-21.20,-175.20,71.60,3.36
1,1,Vaini,75,88,-21.20,-175.20,71.60,3.36
2,2,Rikitea,21,83,-23.12,-134.97,74.01,12.17
3,3,Hermanus,19,58,-34.42,19.23,57.00,1.99
4,4,Shakhtinsk,75,75,49.71,72.59,41.00,6.71
...,...,...,...,...,...,...,...,...
582,582,Taoudenni,0,6,22.68,-3.98,110.12,4.41
583,583,Poputnaya,16,4,44.51,41.44,69.01,17.00
584,584,Havøysund,0,68,71.00,24.66,42.80,18.34
585,585,Dunedin,0,57,-45.87,170.50,41.00,4.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)
locations=weather_df[["Lat","Lng"]]
humidity=weather_df["Humidity"]

#create heat layer
heat_layer=gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,
                              max_intensity=100,point_radius=3)
fig=gmaps.figure(center=(0,0),zoom_level=1)
#add layer
fig.add_layer(heat_layer)
#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow down the cities with wind speed less than 10 mph, cloudiness equals 0 and maxt temp between 65 and 80 

narrowed_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) &
                                   (weather_df["Max Temp"] >= 65) & (weather_df["Max Temp"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Cloudiness,Humidity,Lat,Lng,Max Temp,Wind Speed
86,86,San Quintín,0,74,30.48,-115.95,69.62,3.09
114,114,Topolog,0,39,44.88,28.37,66.20,9.17
284,284,Çermik,0,33,38.14,39.45,77.00,1.12
297,297,Tiznit Province,0,42,29.58,-9.50,78.03,7.52
303,303,Tālcher,0,65,20.95,85.22,79.88,2.53
304,304,Lagoa,0,86,39.05,-27.98,71.19,7.38
378,378,Dārāb,0,17,28.75,54.54,78.01,2.95
510,510,Diego de Almagro,0,18,-26.37,-70.05,74.16,8.61
526,526,Guerrero Negro,0,69,27.98,-114.06,73.22,8.72
552,552,Arbuzynka,0,41,47.91,31.32,67.33,8.95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df=narrowed_city_df.loc[:,["City","Lat","Lng"]]
hotel_df["Hotel Name"]=""
hotel_df

,City,Lat,Lng,Hotel Name
86,San Quintín,30.48,-115.95,
114,Topolog,44.88,28.37,
284,Çermik,38.14,39.45,
297,Tiznit Province,29.58,-9.50,
303,Tālcher,20.95,85.22,
304,Lagoa,39.05,-27.98,
378,Dārāb,28.75,54.54,
510,Diego de Almagro,-26.37,-70.05,
526,Guerrero Negro,27.98,-114.06,
552,Arbuzynka,47.91,31.32,


In [6]:
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 86: San Quintín.
Closest hotel in San Quintín is Old Mill Hotel.
------------
Retrieving Results for Index 114: Topolog.
Closest hotel in Topolog is Pensiunea codru.
------------
Retrieving Results for Index 284: Çermik.
Closest hotel in Çermik is Gıran Park Termal Otel.
------------
Retrieving Results for Index 297: Tiznit Province.
Missing field/result... skipping.
------------
Retrieving Results for Index 303: Tālcher.
Closest hotel in Tālcher is Hotel Surya Garden.
------------
Retrieving Results for Index 304: Lagoa.
Closest hotel in Lagoa is Graciosa Resort - Biosphere Island Hotel.
------------
Retrieving Results for Index 378: Dārāb.
Closest hotel in Dārāb is Naghshe shapoor Hotel.
------------
Retrieving Results for Index 510: Diego de Almagro.
Closest hotel in Diego de Almagro is Hotel Hambyll.
------------
Retrieving Results for Index 526: Guerrero Negro.
Closest hotel in Guerrero Negro is The Halfway Inn.
------------
Retrieving Results for Inde

In [7]:
#drop rows with no results
hotel_df=hotel_df.replace(r'^\s*$', np.NaN, regex=True)
hotel_df.dropna(inplace=True)
hotel_df

,City,Lat,Lng,Hotel Name
86,San Quintín,30.48,-115.95,Old Mill Hotel
114,Topolog,44.88,28.37,Pensiunea codru
284,Çermik,38.14,39.45,Gıran Park Termal Otel
303,Tālcher,20.95,85.22,Hotel Surya Garden
304,Lagoa,39.05,-27.98,Graciosa Resort - Biosphere Island Hotel
378,Dārāb,28.75,54.54,Naghshe shapoor Hotel
510,Diego de Almagro,-26.37,-70.05,Hotel Hambyll
526,Guerrero Negro,27.98,-114.06,The Halfway Inn


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))